In [1]:
import pandas as pd
import numpy as np
import jsonlines
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer

def load_jsonlines(path: str):
    data = []
    with jsonlines.open(path, "r") as reader:
        for items in reader:
            for i in range(len(items["transactions"])):
                items["transactions"][i] = str(items["transactions"][i])
            for i in range(len(items["amounts"])):
                items["amounts"][i] = str(items["amounts"][i])
            data.append(items)
    return data

In [2]:
#!pip install jsonlines

In [3]:
X_train = load_jsonlines("target_clf/train.jsonl")
X_train = pd.DataFrame(X_train)
Y_train = X_train["label"]
del(X_train["label"])
X_train.head()

,transactions,amounts,client_id
0,"[362, 169, 169, 362, 207, 236, 210, 259, 365, ...","[6000, 500, 1000, 1500, 13500, 6000, 2500, 750...",15
1,"[215, 231, 210, 210, 362, 169, 169, 362, 207, ...","[500, 3000, 500, 3500, 6000, 500, 1000, 1500, ...",15
2,"[272, 272, 290, 239, 258, 258, 266, 169, 249, ...","[1000, 1500, 1500, 5000, 4500, 2500, 28500, 50...",15
3,"[272, 272, 290, 239, 258, 258, 266, 169, 249, ...","[1000, 1500, 1500, 5000, 4500, 2500, 28500, 50...",15
4,"[249, 365, 259, 251, 365, 259, 210, 210, 251, ...","[500, 500, 2000, 10500, 2500, 31500, 1500, 500...",15


In [4]:
Y_train.head()

0    2
1    2
2    2
3    2
4    2
Name: label, dtype: int64

In [5]:
Y_train.unique()

array([2, 1, 3, 0])

In [6]:
X_valid = load_jsonlines("target_clf/valid.jsonl")
X_valid = pd.DataFrame(X_valid)
Y_valid = X_valid["label"]
del(X_valid["label"])
X_valid.head()

,transactions,amounts,client_id
0,[402],[3500],588
1,"[402, 253]","[3500, 3500]",588
2,"[402, 253]","[3500, 3500]",588
3,"[402, 253]","[3500, 3500]",588
4,"[402, 253]","[3500, 3500]",588


In [7]:
X_test = load_jsonlines("test.jsonl")
X_test = pd.DataFrame(X_train)
X_test.head()

,transactions,amounts,client_id
0,"[362, 169, 169, 362, 207, 236, 210, 259, 365, ...","[6000, 500, 1000, 1500, 13500, 6000, 2500, 750...",15
1,"[215, 231, 210, 210, 362, 169, 169, 362, 207, ...","[500, 3000, 500, 3500, 6000, 500, 1000, 1500, ...",15
2,"[272, 272, 290, 239, 258, 258, 266, 169, 249, ...","[1000, 1500, 1500, 5000, 4500, 2500, 28500, 50...",15
3,"[272, 272, 290, 239, 258, 258, 266, 169, 249, ...","[1000, 1500, 1500, 5000, 4500, 2500, 28500, 50...",15
4,"[249, 365, 259, 251, 365, 259, 210, 210, 251, ...","[500, 500, 2000, 10500, 2500, 31500, 1500, 500...",15


In [8]:
from sklearn.feature_extraction.text import TfidfTransformer

In [9]:
# working about 3 min on train
# working with one column as X_train.transactions or X_train.amounts


tf = X_train.transactions.apply(lambda x: pd.Series(x).value_counts()).fillna(0)   
tf.sort_index(inplace=True, axis=1)
print("Find these keys:", tf.keys())
    
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(tf)
idf = pd.DataFrame({'feature_name':tf.keys(), 'idf_weights':tfidf_transformer.idf_})
print("idf: ", idf)

tf_idf = pd.DataFrame(X.toarray() ,columns=tf.keys())
print("tf_idf:", tf_idf)
 


Find these keys: Index(['0', '1', '10', '100', '101', '102', '103', '104', '105', '106',
       ...
       '85', '86', '87', '89', '9', '94', '96', '97', '98', '99'],
      dtype='object', length=371)
idf:      feature_name  idf_weights
0              0    10.560089
1              1     8.151916
2             10     9.556787
3            100    11.348547
4            101    12.398369
..           ...          ...
366           94    11.208785
367           96    10.680717
368           97    13.245667
369           98     9.431624
370           99     7.952362

[371 rows x 2 columns]
tf_idf:           0    1   10  100  101  102  103  104  105  106  ...   85   86   87  \
0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4

In [10]:
X = tf_idf

In [11]:
X

,0,1,10,100,101,102,103,104,105,106,...,85,86,87,89,9,94,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
624228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
624229,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
624230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
tf = X_train.amounts.apply(lambda x: pd.Series(x).value_counts()).fillna(0)   
tf.sort_index(inplace=True, axis=1)
print("Find these keys:", tf.keys())
    
tfidf_transformer_amounts = TfidfTransformer()
X1 = tfidf_transformer_amounts.fit_transform(tf)
idf = pd.DataFrame({'feature_name':tf.keys(), 'idf_weights':tfidf_transformer_amounts.idf_})
print("idf: ", idf)

tf_idf = pd.DataFrame(X1.toarray() ,columns=tf.keys())
print("tf_idf:", tf_idf)

Find these keys: Index(['0', '1000', '10000', '100000', '100500', '101000', '101500', '102000',
       '102500', '103000',
       ...
       '9500', '95000', '95500', '96000', '96500', '97000', '97500', '98000',
       '98500', '99000'],
      dtype='object', length=416)
idf:      feature_name  idf_weights
0              0     1.101088
1           1000     1.273394
2          10000     4.236586
3         100000     8.311193
4         100500    11.166225
..           ...          ...
411        97000     9.340333
412        97500    10.494131
413        98000     9.307326
414        98500     9.889932
415        99000     9.800984

[416 rows x 2 columns]
tf_idf:                0      1000  10000  100000  100500  101000  101500  102000  \
0       0.088187  0.254969    0.0     0.0     0.0     0.0     0.0     0.0   
1       0.103589  0.239598    0.0     0.0     0.0     0.0     0.0     0.0   
2       0.114746  0.199053    0.0     0.0     0.0     0.0     0.0     0.0   
3       0.071229  0.24

In [13]:
X1 = tf_idf
X1

,0,1000,10000,100000,100500,101000,101500,102000,102500,103000,...,9500,95000,95500,96000,96500,97000,97500,98000,98500,99000
0,0.088187,0.254969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.103589,0.239598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.084467,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.114746,0.199053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.116956,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.071229,0.247127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.069029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624227,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
624228,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
624229,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
624230,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
X["777"] = X["0"]
del(X["0"])

In [15]:
import pandas as pd
X_final = pd.concat([X, X1], sort = False, axis = 1)

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import svm, datasets
pipe = Pipeline([('classifier' , LogisticRegression())])
# pipe = Pipeline([('classifier', RandomForestClassifier())])

# Create param grid.

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__solver' : ['liblinear']},
]

# Create grid search object
"""
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X, Y_train)
print(clf.score(X, Y_train))
"""

#pipe = Pipeline('lr', LogisticRegression(multi_class='multinomial', tol=1e-3, max_iter=500)
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=1)
clf.fit(X_final, Y_train)
clf_pred = clf.predict(X_final)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  5.5min finished


In [20]:
import joblib

print(clf.score(X_final, Y_train))

#save your model or results
joblib.dump(clf, 'target_logistic_cv_5.pkl')

#load your model for further usage
#joblib.load("model_file_name.pkl")


0.677562188417127


['target_logistic_cv_5.pkl']

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import svm, datasets
##pipe = Pipeline([('classifier' , LogisticRegression())])
"""
pipe = Pipeline([('classifier', RandomForestClassifier(max_depth=17, random_state=0))])

# Create param grid.

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(1, 1, 1),
    'classifier__solver' : ['liblinear']},
]

# Create grid search object
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X, Y_train)
print(clf.score(X, Y_train))
"""
from sklearn.ensemble import RandomForestClassifier
 
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
clf.fit(X_final, Y_train)

"""
pipe = Pipeline('lr', LogisticRegression(multi_class='multinomial', tol=1e-3, max_iter=500)
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)
clf.fit(X_final, Y_train)
clf_pred = clf.predict(X_final)
"""

"\npipe = Pipeline('lr', LogisticRegression(multi_class='multinomial', tol=1e-3, max_iter=500)\nclf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)\nclf.fit(X_final, Y_train)\nclf_pred = clf.predict(X_final)\n"

In [33]:
print(clf.score(X_final, Y_train), " train acc")
print(clf.score(X_valid_final, Y_valid), " valid acc")
#save your model or results
joblib.dump(clf, 'target_forest_dep_50_est_10.pkl')


0.917793384510887  train acc
0.6479165329052969  valid acc


['target_forest_dep_50_est_10.pkl']

In [23]:
tf = X_valid.transactions.apply(lambda x: pd.Series(x).value_counts()).fillna(0)   
tf.sort_index(inplace=True, axis=1)
print("Find these keys:", tf.keys())
tf = pd.DataFrame(tf, columns=X.keys()).fillna(0)  # yes, here we need X.keys from train fitting 
 
X2 = tfidf_transformer.transform(tf)
idf = pd.DataFrame({'feature_name':tf.keys(), 'idf_weights':tfidf_transformer.idf_})
print("idf: ", idf)

tf_idf = pd.DataFrame(X2.toarray() ,columns=tf.keys())
print("tf_idf:", tf_idf)

Find these keys: Index(['0', '1', '10', '100', '101', '103', '105', '109', '11', '110',
       ...
       '80', '81', '82', '83', '84', '87', '9', '96', '98', '99'],
      dtype='object', length=299)
idf:      feature_name  idf_weights
0              1    10.560089
1             10     8.151916
2            100     9.556787
3            101    11.348547
4            102    12.398369
..           ...          ...
366           96    11.208785
367           97    10.680717
368           98    13.245667
369           99     9.431624
370          777     7.952362

[371 rows x 2 columns]
tf_idf:           1   10  100  101  102  103  104  105  106  109  ...   86   87   89  \
0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4 

In [24]:
X2 = tf_idf
X2

,1,10,100,101,102,103,104,105,106,109,...,86,87,89,9,94,96,97,98,99,777
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155748,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
tf = X_valid.amounts.apply(lambda x: pd.Series(x).value_counts()).fillna(0)   
tf.sort_index(inplace=True, axis=1)
print("Find these keys:", tf.keys())
tf = pd.DataFrame(tf, columns=X1.keys()).fillna(0)  # yes, here we need X2.keys() from train fitting

#tfidf_transformer_3 = TfidfTransformer()
X3 = tfidf_transformer_amounts.transform(tf)
idf = pd.DataFrame({'feature_name':tf.keys(), 'idf_weights':tfidf_transformer_amounts.idf_})
print("idf: ", idf)

tf_idf = pd.DataFrame(X3.toarray() ,columns=tf.keys())
print("tf_idf:", tf_idf)

Find these keys: Index(['0', '1000', '10000', '100000', '100500', '101000', '101500', '102000',
       '102500', '103000',
       ...
       '95000', '95500', '96000', '96500', '97000', '97500', '98000', '98500',
       '99000', '99500'],
      dtype='object', length=293)
idf:      feature_name  idf_weights
0              0     1.101088
1           1000     1.273394
2          10000     4.236586
3         100000     8.311193
4         100500    11.166225
..           ...          ...
411        97000     9.340333
412        97500    10.494131
413        98000     9.307326
414        98500     9.889932
415        99000     9.800984

[416 rows x 2 columns]
tf_idf:                0      1000  10000  100000  100500  101000  101500  102000  \
0       0.000000  0.000000    0.0     0.0     0.0     0.0     0.0     0.0   
1       0.000000  0.000000    0.0     0.0     0.0     0.0     0.0     0.0   
2       0.000000  0.000000    0.0     0.0     0.0     0.0     0.0     0.0   
3       0.000000  0.0

In [26]:
X3 = tf_idf
X3

,0,1000,10000,100000,100500,101000,101500,102000,102500,103000,...,9500,95000,95500,96000,96500,97000,97500,98000,98500,99000
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155745,0.863715,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155746,0.895959,0.086347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155747,0.857625,0.090167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155748,0.861185,0.074696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#X2["777"] = X2["0"]
#del(X2["0"])

In [28]:
import pandas as pd
X_valid_final = pd.concat([X2, X3], sort = False, axis = 1)

In [29]:
X_valid_final["1000"].max()

1.0

In [30]:
X_valid_final.max()

1        0.985311
10       0.139350
100      0.941553
101      0.061341
102      0.000000
           ...   
97000    1.000000
97500    0.432920
98000    0.633070
98500    0.587276
99000    0.788747
Length: 787, dtype: float64

In [ ]:
clf.score(X_valid_final, Y_valid)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=2, learning_rate=1.0,
max_depth=20, random_state=0).fit(X_final, Y_train)


In [ ]:
print(clf.score(X_final, Y_train), " train acc")
print(clf.score(X_valid_final, Y_valid), " valid acc")
#save your model or results
joblib.dump(clf, 'target_BOOSTING_dep_20_est_2_lr_1.0.pkl')

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_final, Y_train)
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [ ]:
print(clf.score(X_final, Y_train), " train acc")
print(clf.score(X_valid_final, Y_valid), " valid acc")
#save your model or results
joblib.dump(clf, 'SVC_1.pkl')

In [ ]:
tf = X_test.transactions.apply(lambda x: pd.Series(x).value_counts()).fillna(0)   
tf.sort_index(inplace=True, axis=1)
print("Find these keys:", tf.keys())
tf = pd.DataFrame(tf, columns=X.keys()).fillna(0)  # yes, here we need X.keys from train fitting 
 
X_testing = tfidf_transformer.transform(tf)
idf = pd.DataFrame({'feature_name':tf.keys(), 'idf_weights':tfidf_transformer.idf_})
print("idf: ", idf)

tf_idf = pd.DataFrame(X_testing.toarray() ,columns=tf.keys())
print("tf_idf:", tf_idf)

In [ ]:
X_testing = tf_idf
X_testing

In [ ]:
tf = X_test.amounts.apply(lambda x: pd.Series(x).value_counts()).fillna(0)   
tf.sort_index(inplace=True, axis=1)
print("Find these keys:", tf.keys())
tf = pd.DataFrame(tf, columns=X1.keys()).fillna(0)  # yes, here we need X.keys from train fitting 
 
X_testing_1 = tfidf_transformer.transform(tf)
idf = pd.DataFrame({'feature_name':tf.keys(), 'idf_weights':tfidf_transformer.idf_})
print("idf: ", idf)

tf_idf = pd.DataFrame(X_testing_1.toarray() ,columns=tf.keys())
print("tf_idf:", tf_idf)

In [ ]:
X_testing_1 = tf_idf
X_testing_1

In [ ]:
X_testing["777"] = X_testing["0"]
del(X_testing["0"])